In [1]:
import pandas as pd
import numpy as np
from time import perf_counter, perf_counter_ns
from time import sleep
from datetime import datetime 
from datetime import timedelta
# import datetime
import json
import os
from sys import getsizeof
# import datetime
import random
# from py_vollib.black_scholes.implied_volatility import implied_volatility as bs_iv
# from py_vollib.black_scholes.greeks import analytical as greeks
# import time
# import ks_api_client
from kiteconnect import KiteConnect
# from kiteconnect import KiteTicker

In [85]:
current_datetime = datetime(2021,5,17,15,0)
start_datetime = datetime(2021,5,17,15,15)
end_datetime = datetime(2021,5,20,15,15)
historical_data_folder_name = 'historical data'
fno_folder_name='FNO'
equity_folder_name="Equity"
underlying_name= 'NIFTY'

ltp_to_position_distance = .3


current_datestring = current_datetime.strftime("%Y-%m-%d")

parent = os.path.dirname(os.getcwd())

historical_data_folder_path = os.path.join(\
        parent,historical_data_folder_name)

fno_data_folder_path = os.path.join(\
        historical_data_folder_path,fno_folder_name)

fno_file_paths = [\
    os.path.join(fno_data_folder_path,f) \
    for f in os.listdir(fno_data_folder_path) \
        if os.path.isfile(os.path.join(fno_data_folder_path,f)) \
            & (f.split(".")[0].split("_")[0]==current_datestring)\
                ]

# equity_data_folder_path = os.path.join(\
#     historical_data_folder_path,equity_folder_name)
# equity_file_paths = [\
#     os.path.join(equity_data_folder_path,f) \
#     for f in os.listdir(equity_data_folder_path) \
#         if os.path.isfile(os.path.join(equity_data_folder_path,f)) \
#             & (f.split(".")[0].split("_")[0]==current_datestring)\
#                 ]

# data_file_paths = fno_file_paths
# data_file_paths.extend(equity_file_paths)
# equity_book = pd.concat(map(pd.read_csv,data_file_paths))

# tick_book.rename(columns={'Ticker':'ticker',
#                 'LTP':'ltp',
#                 'BuyPrice':'buy_price',
#                 'SellPrice':'sell_price'},
#                 inplace=True)

# tick_book = equity_book[['ticker','ltp','buy_price',\
#             'sell_price','underlying','strike','call_put',\
#             'expiry_date','timestamp']]
# equity_book = tick_book[\
#     (tick_book['underlying']==underlying_name) | \
#     (tick_book['underlying'].isnull())]
# tick_book['expiry_datetime'] = pd.to_datetime(\
#             tick_book['expiry_date']) + \
#                 timedelta (hours=15, minutes =30)
# tick_book['timestamp'] = pd.to_datetime(\
#             tick_book['timestamp'])
# df_group = tick_book.groupby(['ticker','timestamp'])
# tick_book['duplicate_serial'] = df_group[['ltp']].cumcount()
# tick_book['duplicate_count'] = df_group['ltp'].transform('size')
# del df_group
# tick_book['duplicate_fraction'] = \
#     pd.to_timedelta(tick_book['duplicate_serial'] \
#         / tick_book['duplicate_count'], unit='s')
# tick_book['timestamp'] = tick_book['timestamp'] \
#             + tick_book['duplicate_fraction']

# instruments_book = tick_book[['ticker','underlying',\
#             'strike','call_put','expiry_datetime']]

# tick_book = tick_book[['ticker','ltp','buy_price',\
#             'sell_price','underlying','strike','call_put',\
#             'timestamp']]
# instruments_book.drop_duplicates(inplace=True)


In [179]:
current_datetime = datetime(2021,5,17,15,0)
start_datetime = datetime(2021,5,17,15,15)
end_datetime = datetime(2021,5,20,15,15)
historical_data_folder_name = 'historical data'
fno_folder_name='FNO'
equity_folder_name="Equity"
underlying_name= 'NIFTY'

ltp_to_position_distance = .3

delta = timedelta(days=1)
start_date = current_datetime.date()
datestring_list = []
while start_date <= end_datetime.date():
    # print(start_date)
    datestring_list.append(start_date.strftime("%Y-%m-%d"))
    start_date += delta


parent = os.path.dirname(os.getcwd())

historical_data_folder_path = os.path.join(\
        parent,historical_data_folder_name)

fno_data_folder_path = os.path.join(\
        historical_data_folder_path,fno_folder_name)

equity_data_folder_path = os.path.join(\
    historical_data_folder_path,equity_folder_name)

fno_file_paths = [\
        os.path.join(fno_data_folder_path,f) \
        for f in os.listdir(fno_data_folder_path) \
            if os.path.isfile(os.path.join(fno_data_folder_path,f)) \
                & (f.split(".")[0].split("_")[0] in datestring_list)\
                    ]
fno_file_paths.reverse()

print(fno_file_paths)


equity_file_paths = [\
    os.path.join(equity_data_folder_path,f) \
    for f in os.listdir(equity_data_folder_path) \
        if os.path.isfile(os.path.join(equity_data_folder_path,f)) \
            & (f.split(".")[0].split("_")[0] in datestring_list)\
                ]
equity_file_paths.reverse()
print(equity_file_paths)



    
    

['/Users/hg/Space/Code/BatraTwoPeaksStrategy/historical data/FNO/2021-05-17.csv', '/Users/hg/Space/Code/BatraTwoPeaksStrategy/historical data/FNO/2021-05-18_sample.csv']
['/Users/hg/Space/Code/BatraTwoPeaksStrategy/historical data/Equity/2021-05-17.csv', '/Users/hg/Space/Code/BatraTwoPeaksStrategy/historical data/Equity/2021-05-18.csv']


In [184]:
def size_and_rows(count, df):
    print(f"{count}\tSize: {round(getsizeof(df)/1_000_000,2)} MB, \t {round(df.shape[0]/1_000,0)}k rows, {len(df['underlying'].unique())}")

equity_book = pd.read_csv(equity_file_paths[-1])
equity_book.rename(columns={'Ticker':'ticker',
                        'LTP':'ltp',
                        'BuyPrice':'buy_price',
                        'SellPrice':'sell_price'},
                        inplace=True)
equity_book = equity_book[['ticker','ltp','timestamp']]

equity_book['timestamp'] = pd.to_datetime(\
                equity_book['timestamp'])

start_datetime_ltp = equity_book[(equity_book['timestamp']==start_datetime)]['ltp'].iloc[0]
strike_high = start_datetime_ltp * (1+ltp_to_position_distance)
strike_low = start_datetime_ltp * (1-ltp_to_position_distance)

tick_book = pd.DataFrame()
for equity_file_path, fno_file_path in zip (equity_file_paths,fno_file_paths):
    day_tick_book = pd.concat(map(pd.read_csv,[fno_file_path,equity_file_path]))

    size_and_rows(1,day_tick_book)

    day_tick_book.rename(columns={'Ticker':'ticker',
                            'LTP':'ltp',
                            'BuyPrice':'buy_price',
                            'SellPrice':'sell_price'},
                            inplace=True)
    size_and_rows(2,day_tick_book)

    day_tick_book = day_tick_book[['ticker','ltp','buy_price',\
                        'sell_price','underlying','strike','call_put',\
                        'expiry_date','timestamp']]
    size_and_rows(3,day_tick_book)

    day_tick_book = day_tick_book[\
                (day_tick_book['underlying']==underlying_name) | \
                (day_tick_book['underlying'].isnull())]
    size_and_rows(4,day_tick_book)

    day_tick_book['expiry_datetime'] = pd.to_datetime(\
                        day_tick_book['expiry_date']) + \
                            timedelta (hours=15, minutes =30)
    size_and_rows(5,day_tick_book)

    day_tick_book['timestamp'] = pd.to_datetime(\
                    day_tick_book['timestamp'])
    size_and_rows(6,day_tick_book)

    day_tick_book = day_tick_book[day_tick_book['timestamp']>=current_datetime]
    size_and_rows(7,day_tick_book)

    day_tick_book.sort_values(by=['ticker','timestamp'],inplace=True)
    size_and_rows(8,day_tick_book)

    day_tick_book.drop_duplicates(subset=['ticker','timestamp'],keep='first',inplace=True)
    size_and_rows(9,day_tick_book)

    day_tick_book = day_tick_book[(day_tick_book['expiry_datetime'] == day_tick_book['expiry_datetime'].min()) | \
                        (day_tick_book['expiry_datetime'].isnull())]
    size_and_rows(10,day_tick_book)

    day_tick_book = day_tick_book[(day_tick_book['strike']>=strike_low) & (day_tick_book['strike']<=strike_high)]
    size_and_rows(11,day_tick_book)

    tick_book = pd.concat([tick_book,day_tick_book])



1	Size: 2622.72 MB, 	 4714.0k rows, 2
2	Size: 2622.72 MB, 	 4714.0k rows, 2
3	Size: 1849.65 MB, 	 4714.0k rows, 2
4	Size: 1849.65 MB, 	 4714.0k rows, 2
5	Size: 1887.36 MB, 	 4714.0k rows, 2
6	Size: 1566.82 MB, 	 4714.0k rows, 2
7	Size: 135.15 MB, 	 408.0k rows, 2
8	Size: 135.15 MB, 	 408.0k rows, 2
9	Size: 128.03 MB, 	 386.0k rows, 2
10	Size: 43.62 MB, 	 133.0k rows, 2
11	Size: 42.55 MB, 	 128.0k rows, 1
1	Size: 23.13 MB, 	 47.0k rows, 2
2	Size: 23.13 MB, 	 47.0k rows, 2
3	Size: 15.47 MB, 	 47.0k rows, 2
4	Size: 15.47 MB, 	 47.0k rows, 2
5	Size: 15.85 MB, 	 47.0k rows, 2
6	Size: 12.67 MB, 	 47.0k rows, 2
7	Size: 1.68 MB, 	 7.0k rows, 2
8	Size: 1.68 MB, 	 7.0k rows, 2
9	Size: 1.59 MB, 	 6.0k rows, 2
10	Size: 1.28 MB, 	 5.0k rows, 2
11	Size: 0.21 MB, 	 1.0k rows, 1


In [178]:
a = [1,2,3,4]
b = ['v','b','n','m','g']
a.reverse()
b.reverse()
for e,f in zip(a,b):
    print(e,f)

4 g
3 m
2 n
1 b


In [181]:
day_tick_book

,ticker,ltp,buy_price,sell_price,underlying,strike,call_put,expiry_date,timestamp,expiry_datetime
208357,NIFTY20MAY2112800PE.NFO,0.85,0.85,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:01,2021-05-20 15:30:00
208358,NIFTY20MAY2112800PE.NFO,0.85,0.85,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:02,2021-05-20 15:30:00
208359,NIFTY20MAY2112800PE.NFO,0.85,0.85,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:03,2021-05-20 15:30:00
208360,NIFTY20MAY2112800PE.NFO,0.90,0.80,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:12,2021-05-20 15:30:00
208361,NIFTY20MAY2112800PE.NFO,0.90,0.80,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:13,2021-05-20 15:30:00
...,...,...,...,...,...,...,...,...,...,...
2900966,NIFTY20MAY2116450CE.NFO,0.50,0.50,0.55,NIFTY,16450.0,CE,2021-05-20 00:00:00,2021-05-17 15:29:28,2021-05-20 15:30:00
2900967,NIFTY20MAY2116450CE.NFO,0.50,0.50,0.55,NIFTY,16450.0,CE,2021-05-20 00:00:00,2021-05-17 15:29:40,2021-05-20 15:30:00
2900968,NIFTY20MAY2116450CE.NFO,0.55,0.45,0.55,NIFTY,16450.0,CE,2021-05-20 00:00:00,2021-05-17 15:29:45,2021-05-20 15:30:00
2900969,NIFTY20MAY2116450CE.NFO,0.55,0.45,0.55,NIFTY,16450.0,CE,2021-05-20 00:00:00,2021-05-17 15:29:48,2021-05-20 15:30:00


In [183]:
df=pd.DataFrame()
a = pd.concat([df,day_tick_book])
a

,ticker,ltp,buy_price,sell_price,underlying,strike,call_put,expiry_date,timestamp,expiry_datetime
208357,NIFTY20MAY2112800PE.NFO,0.85,0.85,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:01,2021-05-20 15:30:00
208358,NIFTY20MAY2112800PE.NFO,0.85,0.85,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:02,2021-05-20 15:30:00
208359,NIFTY20MAY2112800PE.NFO,0.85,0.85,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:03,2021-05-20 15:30:00
208360,NIFTY20MAY2112800PE.NFO,0.90,0.80,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:12,2021-05-20 15:30:00
208361,NIFTY20MAY2112800PE.NFO,0.90,0.80,0.90,NIFTY,12800.0,PE,2021-05-20 00:00:00,2021-05-17 15:00:13,2021-05-20 15:30:00
...,...,...,...,...,...,...,...,...,...,...
2900966,NIFTY20MAY2116450CE.NFO,0.50,0.50,0.55,NIFTY,16450.0,CE,2021-05-20 00:00:00,2021-05-17 15:29:28,2021-05-20 15:30:00
2900967,NIFTY20MAY2116450CE.NFO,0.50,0.50,0.55,NIFTY,16450.0,CE,2021-05-20 00:00:00,2021-05-17 15:29:40,2021-05-20 15:30:00
2900968,NIFTY20MAY2116450CE.NFO,0.55,0.45,0.55,NIFTY,16450.0,CE,2021-05-20 00:00:00,2021-05-17 15:29:45,2021-05-20 15:30:00
2900969,NIFTY20MAY2116450CE.NFO,0.55,0.45,0.55,NIFTY,16450.0,CE,2021-05-20 00:00:00,2021-05-17 15:29:48,2021-05-20 15:30:00
